In [45]:
#import das bases *devem estar na mesma pasta do arquivo .ipynb
import pandas as pd

raw_train = pd.read_csv (r'train.csv')
raw_test = pd.read_csv (r'test.csv')


In [46]:
#analise exploratoria das variaveis
print(raw_train.head())
print(raw_train.describe())

print(raw_test.head())
print(raw_test.describe())

#a base test não possui a variavel resposta, portanto será necessario quebrar a base train em treinamento e teste para modelagem

print(raw_train['Survived'].value_counts())
print(raw_train['Survived'].isna().sum())
#2 valores - 0-549 e 1-342, não há valores nulos

print(raw_train['Pclass'].value_counts())
print(raw_train['Pclass'].isna().sum())
#3 valores 3-491 / 1-216 / 2-184, não há valores nulos

print(raw_train['Sex'].value_counts())
print(raw_train['Sex'].isna().sum())
#2 valores - male-577 e female-314, não há valores nulos

print(raw_train['Parch'].value_counts())
print(raw_train['Parch'].isna().sum())
#7 valores, não há valores nulos

print(raw_train['Ticket'].value_counts())
print(raw_train['Ticket'].isna().sum())
#681 valores, esse campo será descartado por não descriminar a base, possui 681 valores unicos em uma base com 891 registros

print(raw_train['Cabin'].value_counts())
print(raw_train['Cabin'].isna().sum())
#147 valores, esse campo será descartado por estar nulo em 687 registros em uma base que contem 891

print(raw_train['Embarked'].value_counts())
print(raw_train['Embarked'].isna().sum())
#3 valores, 2 registros nulos


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  

In [47]:
#data prep e engineering
prep_train=raw_train.copy()
#
print(prep_train.head())
#descartando variaveis para o modelo (name por que não tem poder de descriminação na base)
del prep_train['Name'],prep_train['Ticket'],prep_train['Cabin']

#criacao de variavel categoria a partir de SibSp, no caso dessa variavel e dado o contexo é mais importante saber se ele possuiu parentes no navio ou nao
prep_train['SibSp_cat'] = prep_train['SibSp'].apply(lambda x: 'Yes' if x>=1 else 'No' )
print(prep_train['SibSp_cat'].value_counts())

#criacao de variavel categoria a partir de Parch,no caso dessa variavel e dado o contexo é mais importante saber se ele possuiu filhos no navio ou nao
prep_train['Parch_cat'] = prep_train['Parch'].apply(lambda x: 'Yes' if x>=1 else 'No' )
print(prep_train['Parch_cat'].value_counts())

#descartando as variaveis que foram criadas categorias para elas
del prep_train['Parch'],prep_train['SibSp']

#Substituicao dos valores nulos em idade pela media
prep_train['Age'] = prep_train['Age'].apply(lambda x: 29.7 if x!=x else x )

#Substituicao dos valores nulos em idade pela media
prep_train['Fare'] = prep_train['Fare'].apply(lambda x: 32.20 if x!=x else x )

prep_train['age_cat'] = prep_train['Age'].apply(lambda x: 'children' if x<=14 else ('teenneger' if x<=21 else ('adult'if x<=50 else 'elder')) )
print(prep_train['age_cat'].value_counts())

#criacao de variavel categoria a partir de fare, utilizando os quartis para divisão low 1 quartil, normal 2 e 3 quartil, high 4 quartil
prep_train['fare_cat'] = prep_train['Fare'].apply(lambda x: 'low' if x<=7.91 else ('Normal' if x<=31 else 'High') )
print(prep_train['fare_cat'].value_counts())

#transformação para categoric
prep_train['Pclass'] = prep_train.Pclass.astype(str)

#normalizacao das variaveis continuas
from sklearn import preprocessing
df = prep_train.pop('Age')
d = preprocessing.normalize([df])
prep_train['Age'] = d[0]

df = prep_train.pop('Fare')
d = preprocessing.normalize([df])
prep_train['Fare'] = d[0]

#criacao de dummies para os modelos em que elas são necessarias
prep_train_dummies = prep_train.copy()
prep_train_dummies = pd.get_dummies(prep_train_dummies)


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
No

In [48]:
#Divisao entre treino e teste
import numpy as np
msk = np.random.rand(len(prep_train)) < 0.68 # cria uma mascara com as posicoes

X_train = prep_train[msk]
Y_train = X_train.pop('Survived')
X_test = prep_train[~msk]
Y_test = X_test.pop('Survived')

X_train_dummies = prep_train_dummies[msk]
Y_train_dummies = X_train_dummies.pop('Survived')
X_test_dummies = prep_train_dummies[~msk]
Y_test_dummies = X_test_dummies.pop('Survived')


In [49]:
#MODELAGEM
# será utilizada a acuracia no conjunto de teste como metodo de comparacao dos modelos

In [50]:
# Importacao do modelo
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# Instanciamento do modelo
logreg = LogisticRegression()
# treinamento do modelo
logreg.fit(X_train_dummies, Y_train_dummies)
#predicao no conjunto de teste
y_pred = logreg.predict(X_test_dummies)

print('Acuracia do modelo Regressao Logistica no conjunto de treino: {:.2f}'.format(logreg.score(X_train_dummies, Y_train_dummies)))
print('Acuracia do modelo Regressao Logistica no conjunto de teste: {:.2f}'.format(logreg.score(X_test_dummies, Y_test_dummies)))
#Sem overfitting

Acuracia do modelo Regressao Logistica no conjunto de treino: 0.81
Acuracia do modelo Regressao Logistica no conjunto de teste: 0.76


E:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [51]:
# Importacao do modelo
from sklearn.naive_bayes import GaussianNB
# Instanciamento do modelo
nb = GaussianNB()
# treinamento do modelo
nb.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = nb.predict(X_test_dummies)

print('Acuracia do modelo Naive Bayes no conjunto de teste: {:.2f}'.format(nb.score(X_test_dummies, Y_test_dummies)))
print('Acuracia do modelo Naive Bayes descendente no conjunto de teste: {:.2f}'.format(nb.score(X_test_dummies, Y_test_dummies)))
#Sem overfitting

Acuracia do modelo Naive Bayes no conjunto de teste: 0.70
Acuracia do modelo Naive Bayes descendente no conjunto de teste: 0.70


In [52]:
# Importacao do modelo
from sklearn import linear_model
# Instanciamento do modelo
SGDClf = linear_model.SGDClassifier(max_iter = 1000)
# treinamento do modelo
SGDClf.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = SGDClf.predict(X_test_dummies)

print('Acuracia do modelo Gradiente descendente no conjunto de treino: {:.2f}'.format(SGDClf.score(X_train_dummies, Y_train_dummies)))
print('Acuracia do modelo Gradiente descendente no conjunto de teste: {:.2f}'.format(SGDClf.score(X_test_dummies, Y_test_dummies)))
#Sem overfitting

Acuracia do modelo Gradiente descendente no conjunto de treino: 0.40
Acuracia do modelo Gradiente descendente no conjunto de teste: 0.38


In [53]:
# Importacao do modelo
from sklearn.neighbors import KNeighborsClassifier  
# Instanciamento do modelo
knn = KNeighborsClassifier(n_neighbors = 2)
# treinamento do modelo
knn.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = knn.predict(X_test_dummies)

print('Acuracia do modelo KNeighbors no conjunto de treino: {:.2f}'.format(knn.score(X_train_dummies, Y_train_dummies)))
print('Acuracia do modelo KNeighbors no conjunto de teste: {:.2f}'.format(knn.score(X_test_dummies, Y_test_dummies)))
#Houve overfitting

Acuracia do modelo KNeighbors no conjunto de treino: 0.76
Acuracia do modelo KNeighbors no conjunto de teste: 0.60


In [54]:
# Importacao do modelo
from sklearn import svm  
# Instanciamento do modelo
clf = svm.SVC(kernel='linear')
# treinamento do modelo
clf.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = clf.predict(X_test_dummies)

print('Acuracia do modelo Máquina de vetores de suporte no conjunto de treino: {:.2f}'.format(clf.score(X_train_dummies, Y_train_dummies)))
print('Acuracia do modelo Máquina de vetores de suporte no conjunto de teste: {:.2f}'.format(clf.score(X_test_dummies, Y_test_dummies)))
#Sem overfitting

Acuracia do modelo Máquina de vetores de suporte no conjunto de treino: 0.80
Acuracia do modelo Máquina de vetores de suporte no conjunto de teste: 0.75


In [55]:
# Importacao do modelo
from sklearn.ensemble import RandomForestClassifier
# Instanciamento do modelo
rf = RandomForestClassifier(n_estimators = 10000)
# treinamento do modelo
rf.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = rf.predict(X_test_dummies)

print('Acuracia do modelo Random Forest no conjunto de treino: {:.2f}'.format(rf.score(X_train_dummies, Y_train_dummies)))
print('Acuracia do modelo Random Forest no conjunto de teste: {:.2f}'.format(rf.score(X_test_dummies, Y_test_dummies)))
#Houve overfitting


Acuracia do modelo Random Forest no conjunto de treino: 1.00
Acuracia do modelo Random Forest no conjunto de teste: 0.79


In [56]:
#Retreinando o modelo para evitar o overfitting
rf = RandomForestClassifier(n_estimators = 1000,max_depth=10,min_samples_leaf=3)
# treinamento do modelo
rf.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = rf.predict(X_test_dummies)

print('Acuracia do modelo Random Forest no conjunto de treino: {:.2f}'.format(rf.score(X_train_dummies, Y_train_dummies)))
print('Acuracia do modelo Random Forest no conjunto de teste: {:.2f}'.format(rf.score(X_test_dummies, Y_test_dummies)))

Acuracia do modelo Random Forest no conjunto de treino: 0.90
Acuracia do modelo Random Forest no conjunto de teste: 0.78


In [57]:
#Dado que o Random Forest é o melhor modelo, vamos treinar com toda a base de treino disponivel (891 registros) para predizer a base do desafio

In [58]:
#data prep e engineering na base de teste
prep_test=raw_test.copy()
#
print(prep_test.head())
#descartando variaveis para o modelo (name por que não tem poder de descriminação na base)
del prep_test['Name'],prep_test['Ticket'],prep_test['Cabin']

#criacao de variavel categoria a partir de SibSp, no caso dessa variavel e dado o contexo é mais importante saber se ele possuiu parentes no navio ou nao
prep_test['SibSp_cat'] = prep_test['SibSp'].apply(lambda x: 'Yes' if x>=1 else 'No' )
print(prep_test['SibSp_cat'].value_counts())

#criacao de variavel categoria a partir de Parch,no caso dessa variavel e dado o contexo é mais importante saber se ele possuiu filhos no navio ou nao
prep_test['Parch_cat'] = prep_test['Parch'].apply(lambda x: 'Yes' if x>=1 else 'No' )
print(prep_test['Parch_cat'].value_counts())

#descartando as variaveis que foram criadas categorias para elas
del prep_test['Parch'],prep_test['SibSp']

#Substituicao dos valores nulos em idade pela media
prep_test['Age'] = prep_test['Age'].apply(lambda x: 29.7 if x!=x else x )

#Substituicao dos valores nulos em idade pela media
prep_test['Fare'] = prep_test['Fare'].apply(lambda x: 32.20 if x!=x else x )

prep_test['age_cat'] = prep_test['Age'].apply(lambda x: 'children' if x<=14 else ('teenneger' if x<=21 else ('adult'if x<=50 else 'elder')) )
print(prep_test['age_cat'].value_counts())

#criacao de variavel categoria a partir de fare, utilizando os quartis para divisão low 1 quartil, normal 2 e 3 quartil, high 4 quartil
prep_test['fare_cat'] = prep_test['Fare'].apply(lambda x: 'low' if x<=7.91 else ('Normal' if x<=31 else 'High') )
print(prep_test['fare_cat'].value_counts())

#transformação para categoric
prep_test['Pclass'] = prep_test.Pclass.astype(str)

#normalizacao das variaveis continuas
from sklearn import preprocessing
df = prep_test.pop('Age')
d = preprocessing.normalize([df])
prep_test['Age'] = d[0]

df = prep_test.pop('Fare')
d = preprocessing.normalize([df])
prep_test['Fare'] = d[0]

#criacao de dummies para os modelos em que elas são necessarias
prep_test_dummies = prep_test.copy()
prep_test_dummies = pd.get_dummies(prep_test_dummies)


   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
No     283
Yes    135
Name: SibSp_cat, dtype: int64
No     324
Yes     94
Name: Parch_cat, dtype: int64
adult        301
teenneger     56
elder         31
children      30
Name:

In [59]:
X_train_dummies = prep_train_dummies.copy()
Y_train_dummies = X_train_dummies.pop('Survived')
X_test_dummies = prep_test_dummies

In [60]:
# Importacao do modelo
from sklearn.ensemble import RandomForestClassifier
# Instanciamento do modelo
rf = RandomForestClassifier(n_estimators = 1000,max_depth=10,min_samples_leaf=3)
# treinamento do modelo
rf.fit(X_train_dummies, Y_train_dummies);
#predicao no conjunto de teste
y_pred = rf.predict(X_test_dummies)


In [61]:
df_f = X_test_dummies[['PassengerId']].copy()
df_f['Survived'] =y_pred
print(df_f)
df_f.to_csv('titanic_ml.csv',index=False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
